# Проект "Проверка гипотез" #
Цель проекта: проверка изменений направленных на увелечение выручки интернет-магазина

# Описание данных #
1. Файл "/datasets/hypothesis.csv":
 Hypothesis — краткое описание гипотезы;
 Reach — охват пользователей по 10-балльной шкале;
 Impact — влияние на пользователей по 10-балльной шкале;
 Confidence — уверенность в гипотезе по 10-балльной шкале;
 Efforts — затраты ресурсов на проверку гипотезы по 10-балльной шкале. Чем больше значение Efforts, тем дороже проверка гипотезы.
2. Файл "/datasets/orders.csv":
 transactionId — идентификатор заказа;
 visitorId — идентификатор пользователя, совершившего заказ;
 date — дата, когда был совершён заказ;
 revenue — выручка заказа;
 group — группа A/B-теста, в которую попал заказ.
3. Файл "/datasets/visitors.csv":
 date — дата;
 group — группа A/B-теста;
 visitors — количество пользователей в указанную дату в указанной группе A/B-теста

# План проведения исследования #
1. загрузка данных
2. предобработка данных
3. приоритезация гипотез по ICE
4. приоритезация гипотез по RICE
5. A/B-тест
6. анализ результатов A/B-теста

<a id='import'></a>
# Импорт библиотек #

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from IPython.core.display_functions import display
import plotly.express as px
from scipy import stats as st

<a id='data'></a>
# Загрузка данных #

In [2]:
hypothesis = pd.read_csv(filepath_or_buffer='datasets/hypothesis.csv', sep=',')
orders = pd.read_csv(filepath_or_buffer='datasets/orders.csv', sep=',')
visitors = pd.read_csv(filepath_or_buffer='datasets/visitors.csv', sep=',')

<a id='preview_hypothesis'></a>
## Предварительный осмотр hypothesis ##

In [3]:
display(hypothesis)

,Hypothesis,Reach,Impact,Confidence,Efforts
0,"Добавить два новых канала привлечения трафика,...",3,10,8,6
1,"Запустить собственную службу доставки, что сок...",2,5,4,10
2,Добавить блоки рекомендаций товаров на сайт ин...,8,3,7,3
3,"Изменить структура категорий, что увеличит кон...",8,3,3,8
4,"Изменить цвет фона главной страницы, чтобы уве...",3,1,1,1
5,"Добавить страницу отзывов клиентов о магазине,...",3,2,2,3
6,Показать на главной странице баннеры с актуаль...,5,3,8,3
7,Добавить форму подписки на все основные страни...,10,7,8,5
8,"Запустить акцию, дающую скидку на товар в день...",1,9,9,5


In [4]:
hypothesis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Hypothesis  9 non-null      object
 1   Reach       9 non-null      int64 
 2   Impact      9 non-null      int64 
 3   Confidence  9 non-null      int64 
 4   Efforts     9 non-null      int64 
dtypes: int64(4), object(1)
memory usage: 488.0+ bytes


<a id='preview_orders'></a>
## Предварительный осмотр orders ##

In [5]:
display(orders)

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B
...,...,...,...,...,...
1192,2662137336,3733762160,2019-08-14,6490,B
1193,2203539145,370388673,2019-08-14,3190,A
1194,1807773912,573423106,2019-08-14,10550,A
1195,1947021204,1614305549,2019-08-14,100,A


In [6]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   transactionId  1197 non-null   int64 
 1   visitorId      1197 non-null   int64 
 2   date           1197 non-null   object
 3   revenue        1197 non-null   int64 
 4   group          1197 non-null   object
dtypes: int64(3), object(2)
memory usage: 46.9+ KB


## Предварительный осмотр visitors ##

In [7]:
display(visitors)

,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756
...,...,...,...
57,2019-08-27,B,720
58,2019-08-28,B,654
59,2019-08-29,B,531
60,2019-08-30,B,490


In [8]:
visitors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      62 non-null     object
 1   group     62 non-null     object
 2   visitors  62 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB


## Результаты предварительного осмотра данных ##
1. hypothesis:
  - имена столбцов перевести в нижний регистр
  - hypothesis['hypothesis'] - перевести значение в нижний регистр
2. orders:
  - переименовать "transactionId" -> "transaction_id"
  - переименовать "visitorId" -> "visitor_id"
  - "date".type(object) - > "date".type(datetime)
3. visitors:
  - "date".type(object) -> "date".type(datetime)

<a id='prepare'></a>
# Предобработка данных #

<a id='prepare_rename_columns'></a>
## Переименоваине столбцов ##

In [9]:
def prepare_columns(dataframe):
    dataframe.columns = dataframe.columns.str.lower()
    dataframe.columns = dataframe.columns.str.strip()

In [10]:
orders = orders.rename(columns={'transactionId':'transaction_id',
                                'visitorId':'visitor_id'})

hypothesis.columns = hypothesis.columns.str.lower().str.strip()
orders.columns = orders.columns.str.lower().str.strip()
visitors.columns = visitors.columns.str.lower().str.strip()

<a id='prepare_data'></a>
## Изменение данных ##

In [11]:
hypothesis['hypothesis'] = hypothesis['hypothesis'].str.strip().str.lower()

<a id='prepare_type'></a>
## Изменение типов данных ##

In [12]:
orders['date'] = orders['date'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
visitors['date'] = visitors['date'].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

<a id='prepare_dupl'></a>
## Проверка наличия дубликатов ##

In [13]:
print(hypothesis.duplicated().sum())
print(orders.duplicated().sum())
print(visitors.duplicated().sum())

0
0
0


<a id='hypothesis'></a>
# Приоритезация гипотез #

<a id='hypothesis_ice'></a>
## Фреймворк ICE ##

In [14]:
hypothesis['ICE'] = (hypothesis['impact'] * hypothesis['confidence']) / hypothesis['efforts']
hypothesis['RICE'] = (hypothesis['reach'] * hypothesis['impact'] * hypothesis['confidence']) / hypothesis['efforts']
print(hypothesis[['hypothesis', 'ICE', 'RICE']].sort_values(by=['ICE', 'RICE'], ascending=False))

                                          hypothesis        ICE   RICE
8  запустить акцию, дающую скидку на товар в день...  16.200000   16.2
0  добавить два новых канала привлечения трафика,...  13.333333   40.0
7  добавить форму подписки на все основные страни...  11.200000  112.0
6  показать на главной странице баннеры с актуаль...   8.000000   40.0
2  добавить блоки рекомендаций товаров на сайт ин...   7.000000   56.0
1  запустить собственную службу доставки, что сок...   2.000000    4.0
5  добавить страницу отзывов клиентов о магазине,...   1.333333    4.0
3  изменить структура категорий, что увеличит кон...   1.125000    9.0
4  изменить цвет фона главной страницы, чтобы уве...   1.000000    3.0


## Итоги приоритезации гипотез ##
1. различия в оценках гипотез ICE и RICE обусловлены использованием компоненты 'REACH'(охват) в фреймфорке RICE.
   В фреймфорке ICE указанный  компонент не учитывается.

<a id='abtest'></a>
# Анализ A/B-теста #

<a id='abtest_cum_revenue'></a>
## Кумулятивная выручка групп ##

In [15]:
dates_groups = orders[['date', 'group']].drop_duplicates()

### Агрегирование данных о покупателях/заказах/выручке групп, на указанную дату ###

In [16]:
orders_agg = dates_groups.apply(lambda x: orders[np.logical_and(orders['date'] <= x['date'],
                                                                orders['group'] == x['group'])]\
                                .agg({'date':np.max,
                                      'group':np.max,
                                      'transaction_id':pd.Series.nunique,
                                      'visitor_id':pd.Series.nunique,
                                      'revenue':np.sum}), axis='columns')\
    .sort_values(by=list(dates_groups.columns))

### Агрегирование данных о количестве посещений групп, на указанную дату ###

In [17]:
visitors_agg = dates_groups.apply(lambda x: visitors[np.logical_and(visitors['date'] <= x['date'],
                                                                    visitors['group'] == x['group'])]\
                                  .agg({'date':np.max,
                                        'group':np.max,
                                        'visitors':np.sum}), axis='columns')\
    .sort_values(by=list(dates_groups.columns))

### Объединение агрегированных таблиц заказов и посещений для формирования таблицы кумулятивных данных ###

In [18]:
cum_data = orders_agg.merge(right=visitors_agg,
                            how='inner',
                            right_on=['date', 'group'],
                            left_on=['date', 'group'])
cum_data.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

### График кумулятивной выручки групп ###

In [19]:
fig = px.line(data_frame=cum_data,
              x='date',
              y='revenue',
              color='group',
              title='Кумулятивная выручка групп',
              template='seaborn',
              labels={'revenue':'Выручка',
                      'date':'Дата',
                      'group':'Группа'})
fig.show()

## Итоги анализа кумулятивной выручки групп ##
1. кумулятивная выручка группв B растет быстрее кумулятивной выручки группы A
2. 19 августа наблюдается резкий рост кумулятивной выручки в группе B
   Возможные причины:
   - крупные покупки пользователей из группы B
   - большое количество заказов пользователей группы B

<a id='abtest_cum_reciept_mean'></a>
## Кумулятивный средний чек групп ##

In [23]:
fig = px.line(data_frame=cum_data,
              x='date',
              y=cum_data['revenue'] / cum_data['orders'],
              color='group',
              title='Кумулятивный средний чек групп',
              template='seaborn',
              labels={'y':'Средняя сумма чека',
                      'date':'Дата',
                      'group':'Группа'})
fig.show()

## Итоги анализа кумулятивного среднего чека групп ##
1. средний чек пользователей группы B выше чем в группе А
2. 19 августа наблюдается резкий рост среднего чека группы B
   Возможные причины:
   - крупные покупки пользователей из группы B 19 августа

<a id='abtest_cum_reciept_mean_diff'></a>
## Относительное изменение кумулятивного среднего чека групп ##

In [38]:
cum_revenue_a = cum_data[cum_data['group'] == 'A'][['date', 'revenue', 'orders']]
cum_revenue_a['reciept_mean'] = cum_revenue_a['revenue'] / cum_revenue_a['orders']

cum_revenue_b = cum_data[cum_data['group'] == 'B'][['date', 'revenue', 'orders']]
cum_revenue_b['reciept_mean'] = cum_revenue_b['revenue'] / cum_revenue_b['orders']

merge_cum_revenue = cum_revenue_a.merge(right=cum_revenue_b,
                                        how='left',
                                        left_on='date',
                                        right_on='date',
                                        suffixes=['A','B'])
merge_cum_revenue['cum_reciept_diff'] = merge_cum_revenue['reciept_meanB'] / merge_cum_revenue['reciept_meanA'] - 1

### График относительного изменения значения кумулятивного среднего чека групп ###

In [39]:
fig = px.line(data_frame=merge_cum_revenue,
              x='date',
              y='cum_reciept_diff',
              template='seaborn',
              labels={'cum_reciept_diff':'Относительное значение среднего чека',
                      'date':'Дата'},
              title='Относительное изменение значения среднего чека группы B')
fig.add_shape(type='line',
              line_color='salmon',
              x0=merge_cum_revenue['date'].min(),
              x1=merge_cum_revenue['date'].max(),
              y0=0,
              y1=0,
              line_width=3)
fig.show()

## Итоги анализа графика относительного изменения среднего чека группы B ##
1. до 2 августа сумма среднего чека была ниже чем в группе А
2. со второго по 11 августа сумма среднего чека группы B превышала сумму среднего чека группы А.
   Пиковое значение превышения суммы чека, в указанный период зафиксировано 8 и 6 августа.
   Разница в суммах среднего чека 6 и 8 августа составила 35%.
3. В период с 11 по 15 августа сумма среднего чека группы была чем у группы А.
   Пиковое значение падения суммы среднего чека группы B зафиксировано 13 августа.
   Падение составило 13%.
4. 15 августа сумма средних чеков в обеих группах сравнялась.
   До 18 августа наблюдается медленный рост суммы среднего чека группы B.
5. 19 августа зафиксирован резкий рост суммы среднего чека группы B.
   Рост составил 49%.
6. Необходимо провести анализ выбросов средней суммы чека в обеих группах.

<a id='abtest_cum_conversion'></a>
## Кумулятивная конверсия групп ##

In [41]:
fig = px.line(data_frame=cum_data,
              x='date',
              y=cum_data['buyers'] / cum_data['visitors'],
              color='group',
              title='Кумулятивная конверсия групп',
              template='seaborn',
              labels={'y':'Конверсия',
                      'date':'Дата',
                      'group':'Группа'})
fig.show()

## Итоги анализа конверсии групп ##
1. до 18 августа наблюдаются колебания значения конверсии для обеих групп
2. 18 августа конверсия обеих групп выравнивается.
3. конверсия в группе B выше конверсии в группе А ~0.3%

<a id='abtest_cum_conversion_diff'></a>
## Относительное изменение конверсии групп ##